In [4]:
## You must have jupyter-matplotlib (aka ipympl) installed
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt

## For displaying and controlling the app
from ipywidgets import AppLayout, FloatSlider, widgets
from IPython.display import HTML, display, FileLink
import tabulate

from matplotlib.path import Path as mpl_path
def inside_poly(data, vertices):
    """ This is a simple implementation of the 'ray-tracing' method for points inside polygons """
    return mpl_path(vertices).contains_points(data)

In [15]:
## This is the class that works the magic. Ideally place it in a separate non-ipynb file
class LineBuilder:
    def __init__(self, line):
        ## The class is initialized on a empty plot with style attributes only
        self.line = line
        self.xs = list(line.get_xdata())
        self.ys = list(line.get_ydata())
        self.cid = line.figure.canvas.mpl_connect('button_press_event', self)
        
        ## Ipython output fields
        self.out = widgets.Output(layout={'border': '1px solid grey',
                                          'max_width': '80 px',
                                          'min_width': '50 px'
                                         })
        
        self.controls = widgets.Output(layout={'border': '1px solid grey',
                                          'max_width': '80 px',
                                          'min_width': '50 px'
                                         })
        
        with self.out:
            self.dh = display(display_id=True) 
        with self.controls:
            self.button1 = widgets.Button(description="Export")
            self.button2 = widgets.Button(description="Clear")
            self.button3 = widgets.Button(description="Give Kudos")
            display(self.button1)
            display(self.button2)
            display(self.button3)
            self.button1.on_click(self._export)
            self.button2.on_click(self._clear)
            self.button3.on_click(self._kudos)

    def _export(self, b):
        table = np.array([self.xs, self.ys]).T
        np.savetxt('polygon.csv', table, delimiter=',', header="X, Y", comments='')
        local_file = FileLink('./polygon.csv', result_html_prefix="Click here to download: ")
        with self.controls:
            display(local_file)
    
    def _clear(self, b):
        self.xs = []
        self.ys = []
        self.line.set_data(self.xs, self.ys)
        self.line.figure.canvas.draw()
        self._updatetable()
        
    def _kudos(self, b):
        with self.out:
            self.dh.display('Kudos! ')
            
    def _updatetable(self):
        table = np.array([self.xs, self.ys]).T
        with self.out:
            self.out.clear_output()
            self.dh.display(HTML(tabulate.tabulate(table, tablefmt='html'))) 
            
    def __call__(self, event):
        if event.inaxes!=self.line.axes: return
        self.xs.append(event.xdata)
        self.ys.append(event.ydata)
        self.line.set_data(self.xs, self.ys)
        self.line.figure.canvas.draw()
        self._updatetable()
        
    def get_mask(self):
        """ You can also just get the polygon without saving it via the export button"""
        return(np.array([self.xs + [self.xs[0]], self.ys + [self.ys[0]] ]).T)

In [16]:
plt.ioff() ## Needed to avoid double output

## A plot with some nice data
fig = plt.figure(figsize=(5,5))
x = np.random.rand(100)
y = np.random.rand(100)
plt.plot(x, y, 'o')

line, = plt.plot([], [], 'ro-') ## The style of the polygon drawing is set by this empty line object
linebuilder = LineBuilder(line) # Initialized the selection class

# Add the selection output and controls to the ipympl App
AppLayout(
    center=fig.canvas,
    right_sidebar=linebuilder.out,
    left_sidebar=linebuilder.controls,
    pane_widths=[0.2, 0.7, 0.2],
    height='600px',
    grid_gap="30px") 

AppLayout(children=(Output(layout=Layout(border='1px solid grey', grid_area='left-sidebar', max_width='80 px',…

In [17]:
# Access polygon you just built and feed it to a selection function
mypoly = linebuilder.get_mask()
print(mypoly)
inside = inside_poly(np.c_[x, y], mypoly)

# Check if selection matches what you made interactively
plt.ion()
fig = plt.figure(figsize=(5,5))
plt.plot(x, y, 'o')
plt.plot(x[inside], y[inside], 'o', label='Inside')
plt.plot(mypoly[:,0], mypoly[:,1], 'r-', label='Selection')
plt.legend()

[[0.18351542 0.12449863]
 [0.34106576 0.88793416]
 [0.78276939 0.68673177]
 [0.79402299 0.55920914]
 [0.73775501 0.26732399]
 [0.48736251 0.11429682]
 [0.18351542 0.12449863]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …